# Digital Twin FastAPI Backend: Interactive API Demo

This notebook demonstrates how to interact with the digital twin backend using Python. It covers simulation, control, state retrieval, session-based endpoints, and basic integration testing.

## 1. Import Required Libraries

Import Python libraries such as `requests` and others needed for API interaction and data handling.

In [ ]:
# Import required libraries
import httpx
import time
import json
from pprint import pprint

: 

## 2. Start FastAPI App in Notebook

You can start the FastAPI app using Uvicorn in a separate terminal or subprocess. For demonstration, ensure the app is running at `http://localhost:8000` before proceeding.

In [ ]:
# Optionally, start FastAPI app using subprocess (uncomment to use)
# import subprocess
# uvicorn_process = subprocess.Popen([
#     "poetry", "run", "uvicorn", "app.main:app", "--reload", "--host", "0.0.0.0", "--port", "8000"
# ])
# time.sleep(2)  # Wait for server to start
# print("FastAPI app started.")

# For this demo, ensure the app is running at http://localhost:8000

## 3. Interact with Simulation API Endpoints

We will use the `requests` library to interact with the simulation and control endpoints.

In [ ]:
# Define the API base URL
API_URL = "http://localhost:8000"

# Helper function to pretty print responses
def print_response(response):
    print(f"Status code: {response.status_code}")
    try:
        pprint(response.json())
    except Exception:
        print(response.text)

# Create a reusable HTTPX client
client = httpx.Client()

## 4. Simulate Water Tank System

Let's create a session and step the water tank simulation.

In [ ]:
# Create a session for the water tank model
session_init_resp = client.post(f"{API_URL}/simulate/init", json={"model_name": "water_tank"})
print("Session Init Response:")
print_response(session_init_resp)
session_id = session_init_resp.json()["session_id"]

# Step the simulation
step_resp = client.post(f"{API_URL}/simulate/step", json={
    "session_id": session_id,
    "control_input": 5.0,
    "delta_time": 1.0
})
print("\nStep Response:")
print_response(step_resp)

## 5. Simulate Room Temperature System

Now let's create a session and step the room temperature simulation.

In [ ]:
# Create a session for the room temperature model
rt_session_init_resp = client.post(f"{API_URL}/simulate/init", json={"model_name": "room_temperature"})
print("Room Temperature Session Init Response:")
print_response(rt_session_init_resp)
rt_session_id = rt_session_init_resp.json()["session_id"]

# Step the simulation
rt_step_resp = client.post(f"{API_URL}/simulate/step", json={
    "session_id": rt_session_id,
    "control_input": 2.0,
    "delta_time": 1.0
})
print("\nRoom Temperature Step Response:")
print_response(rt_step_resp)

## 6. Read System State

Retrieve and display the current state of each system using the session-based endpoints.

In [ ]:
# Get state for water tank session
state_resp = client.get(f"{API_URL}/simulate/state/{session_id}")
print("Water Tank State:")
print_response(state_resp)

# Get state for room temperature session
rt_state_resp = client.get(f"{API_URL}/simulate/state/{rt_session_id}")
print("Room Temperature State:")
print_response(rt_state_resp)

## 7. Test Session-Based Endpoints

Demonstrate session-based simulation by making multiple requests and showing state persistence.

In [ ]:
# Step the water tank simulation multiple times and show state persistence
for i in range(3):
    step_resp = client.post(f"{API_URL}/simulate/step", json={
        "session_id": session_id,
        "control_input": 2.0 + i,
        "delta_time": 1.0
    })
    print(f"\nStep {i+1} Response:")
    print_response(step_resp)
    state_resp = client.get(f"{API_URL}/simulate/state/{session_id}")
    print(f"Water Tank State after step {i+1}:")
    print_response(state_resp)

## 8. Add Logging and Monitoring Example

Show how to retrieve logs for a session. (Metrics/monitoring can be added if available in the backend.)

In [ ]:
# Retrieve logs for the water tank session
logs_resp = client.get(f"{API_URL}/simulate/logs/{session_id}")
print("Water Tank Session Logs:")
print_response(logs_resp)

## 9. API Integration Test Example

Write a simple integration test in the notebook to verify endpoint responses and state changes.

In [ ]:
# Simple integration test: create, step, and check state

def integration_test():
    # Create session
    resp = client.post(f"{API_URL}/simulate/init", json={"model_name": "water_tank"})

    assert resp.status_code == 200
    
    sid = resp.json()["session_id"]
    # Step
    resp = client.post(f"{API_URL}/simulate/step", json={
        "session_id": sid,
        "control_input": 3.0,
        "delta_time": 1.0
    })
    
    assert resp.status_code == 200
    
    # Get state
    resp = client.get(f"{API_URL}/simulate/state/{sid}")
    
    assert resp.status_code == 200
    
    state = resp.json()["state"]
    print("Integration test state:", state)
    
    assert "level" in state

integration_test()